
# RADOLAN download & upload

This notebook downloads RADOLAN data for all of Germany for a given time period from the [DWD open data server](https://opendata.dwd.de/).  
The data is then processed to a (cf-compliant) netCDF file, which is saved to the `/data` folder.

In the second part of this notebook, the metadata for the RADOLAN data is created in metacatalaog and the netCDF file is linked and uploaded.


## 1.) RADOLAN download

In this part, recent RADOLAN data is downloaded from the DWD open data server using the pacakge `wetterdienst`.  
The data file is then processed with the package `wradlib` and loaded into Python.  
The data is then loaded as an `xarray` dataset. By adding metadata, a cf-compliant netCDF dataset is then saved to the `/data` folder.

In [1]:
# load dependencies
from wetterdienst.provider.dwd.radar import DwdRadarValues, DwdRadarParameter, DwdRadarResolution
import wradlib as wrl
import xarray as xr
from cftime import date2num
import numpy as np
from datetime import datetime as dt
from datetime import timedelta as td


In [2]:
# function to add RADOLAN metadata which makes the netCDF file cf-conform (v1.0)

def add_metadata(data: xr.Dataset, attrs: list, time_units) -> xr.Dataset:
    """
    netCDF.dataset_compliance() (cf-python)
    """
    # global attributes (metadata source: https://opendata.dwd.de/climate_environment/CDC/grids_germany/daily/radolan/recent/bin/DESCRIPTION_gridsgermany-daily-radolan-recent-bin_en.pdf)
    data.attrs['Conventions'] = 'CF-1.10'
    data.attrs['title'] = 'DWD Climate Data Center (CDC): Recent hourly sliding RADOLAN grid of daily precipitation'
    data.attrs['institution'] = 'Deutscher Wetterdienst (DWD)DWD Climate Data Center (CDC)'
    data.attrs['source'] = f"RADOLAN version: {attrs['radolanversion']}. The routine procedure RADOLAN (Radar-Online-Aneichung) \n provides area-wide, spatially and temporally highly resolved quantitative precipitation data in \n eal-time operation for Germany from the combination of the hourly values measured at the precipitation \n stations with the precipitation recording of the 17 weather radars."
    data.attrs['history'] = f"{str(dt.utcnow())}: data download and file creation with xarray / python'"
    data.attrs['references'] = "Bartels, H. et al., 2004: Zusammenfassender Abschlussbericht zum Projekt RADOLAN \n Winterrath T. et al., 2012: On the DWD quantitative precipitation analysis and nowcasting system for real-time application in German flood risk management. Weather Radar and Hydrology, IAHS Publ. 351"
    data.attrs['comment'] = "**DATA ORIGIN:** Weather Radars can only measure the reflected signals from the hydrometeors in the atmosphere and not the precipitation directly. For \n the radarbased quantitative precipitation estimation the radar data are adjusted with the measurements oft he conventional precipitation \n stations. The adjusted radar data is a combination of the two sources of radar and surface stations and therefore these data are using \n the advantages of both data sets. \n **VALIDATION AND UNCERTAINTY ESTIMATE:** Verification of the data from 2013 till 2016 against the daily measurements of the precipitation stations shows a mean median of the \n absolute daily deviations of 0.761 mm/day. This is quite better than the corresponding value of 2.390 mm/day for the non adjusted radar data. \n **ADDITIONAL INFORMATION:** The data are not measured values, but represent a best estimate of precipitation due to the indirect method of radar measurement, \n which is calibrated (quantified) on the measured values of conventional stations (average station distance: approx. 20 km)."

    data.attrs['id'] = "urn:x-wmo:md:de.dwd.cdc::gridsgermany-daily-radolan-recent-bin"

    # variable attributes
    data.radolan.attrs['standard_name'] = 'precipitation_amount'
    data.radolan.attrs['long_name'] = 'Recent hourly sliding RADOLAN grid of daily precipitation'
    data.radolan.attrs['units'] = '1e-4 m'
    data.radolan.attrs['missing_value'] = attrs['nodataflag']

    # coordinate variables
    data.x_dist.attrs['axis'] = 'X'
    data.x_dist.attrs['long_name'] = "RADOLAN Grid x coordinate of projection"
    data.x_dist.attrs['standard_name'] = "projection_x_coordinate"
    data.x_dist.attrs['units'] = "km"

    data.y_dist.attrs['axis'] = 'Y'
    data.y_dist.attrs['long_name'] = "RADOLAN Grid y coordinate of projection"
    data.y_dist.attrs['standard_name'] = "projection_y_coordinate"
    data.y_dist.attrs['units'] = "km"

    data.lon.attrs['standard_name'] = 'longitude'
    data.lon.attrs['long_name'] = 'longitude'
    data.lon.attrs['units'] = 'degrees_east'

    data.lat.attrs['standard_name'] = 'latitude'
    data.lat.attrs['long_name'] = 'latitude'
    data.lat.attrs['units'] = 'degrees_north'

    data.time.attrs['axis'] = 'T'
    data.time.attrs['standard_name'] = 'time'
    data.time.attrs['long_name'] = 'time'
    data.time.encoding['calendar'] = 'gregorian'
    data.time.attrs['units'] = time_units

    # cf convention checker: https://pumatest.nerc.ac.uk/cgi-bin/cf-checker.pl
    
    return data

In [3]:
# function to download RADOLAN data, add metadata and save as netCDF to the given path (if given)


def download_radolan_data(start_date, end_date, path=None, mask=False):
    """
    Download RADOLAN CDC data in daily resolution. 
    """
    #daily_data = {}
    tstamps = []
    data_chunks = []
    attributes_list = []

    radar = DwdRadarValues(
        parameter=DwdRadarParameter.RADOLAN_CDC,
        resolution=DwdRadarResolution.DAILY,
        start_date=start_date,
        end_date=end_date
    )

    for item in radar.query():
        # Decode data using wradlib.
        data, attributes = wrl.io.read_radolan_composite(item.data)

        # Get grid.
        radolan_grid_xy = wrl.georef.get_radolan_grid(
            attributes['nrow'], attributes['ncol'], wgs84=False)
        # Get coordinates (lat, lon).
        radolan_grid_latlon = wrl.georef.get_radolan_grid(
            attributes['nrow'], attributes['ncol'], wgs84=True)

        # Mask data.
        if mask:
            data = np.ma.masked_equal(data, -9999)
        else:
            pass

        # append datetime
        tstamps.append(attributes['datetime'])

        # append date
        data_chunks.append(data)

        # append attributes
        attributes_list.append(attributes)

    # time cf format
    time_units = f"days since {tstamps[0]}"
    time_values = date2num(tstamps, time_units)

    # build xarray DataArray
    data = xr.Dataset(data_vars={'radolan': (['time', 'x', 'y'], data_chunks)},
                      coords={'time': time_values,
                              'x_dist': (['x', 'y'], radolan_grid_xy[:, :, 1]),
                              'y_dist': (['x', 'y'], radolan_grid_xy[:, :, 0]),
                              'lat': (['x', 'y'], radolan_grid_latlon[:, :, 1]),
                              'lon': (['x', 'y'], radolan_grid_latlon[:, :, 0])})

    # add cf metadata to netCDF
    data = add_metadata(data, attributes_list[0], time_units)

    # export / return
    if path:
        data.to_netcdf(path=path, mode='w', format='NETCDF4')
    else:
        return data


**Give time period here**

In [4]:
start_date = dt.now() - td(days=2)
end_date = dt.now()

In [5]:
# execute functions / download and save data
data = download_radolan_data(start_date=start_date, end_date=end_date)#, path='data/radolan.nc')

download_radolan_data(start_date=start_date, end_date=end_date, path='data/radolan.nc')

  0%|          | 0/240 [00:00<?, ?it/s]/home/alexander/anaconda3/envs/wradlib/lib/python3.10/site-packages/dateparser/utils/__init__.py:130: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = tz.localize(date_obj)
/home/alexander/anaconda3/envs/wradlib/lib/python3.10/site-packages/dateparser/utils/__init__.py:130: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = tz.localize(date_obj)
  0%|          | 0/240 [00:00<?, ?it/s]/home/alexander/anaconda3/envs/wradlib/lib/python3.10/site-packages/dateparser/utils/__init__.py:130: PytzUsageWarning: The 

In [6]:
xr.open_dataset("data/radolan.nc")

<xarray.Dataset>
Dimensions:  (time: 47, x: 900, y: 900)
Coordinates:
  * time     (time) datetime64[ns] 2022-09-26T14:50:00 ... 2022-09-28T12:50:00
    x_dist   (x, y) float64 ...
    y_dist   (x, y) float64 ...
    lat      (x, y) float64 ...
    lon      (x, y) float64 ...
Dimensions without coordinates: x, y
Data variables:
    radolan  (time, x, y) float64 ...
Attributes:
    Conventions:  CF-1.10
    title:        DWD Climate Data Center (CDC): Recent hourly sliding RADOLA...
    institution:  Deutscher Wetterdienst (DWD)DWD Climate Data Center (CDC)
    source:       RADOLAN version: 2.29.1. The routine procedure RADOLAN (Rad...
    history:      2022-09-28 13:32:56.557594: data download and file creation...
    references:   Bartels, H. et al., 2004: Zusammenfassender Abschlussberich...
    comment:      **DATA ORIGIN:** Weather Radars can only measure the reflec...
    id:           urn:x-wmo:md:de.dwd.cdc::gridsgermany-daily-radolan-recent-bin

In [7]:
data

<xarray.Dataset>
Dimensions:  (time: 47, x: 900, y: 900)
Coordinates:
  * time     (time) float64 0.0 0.04167 0.08333 0.125 ... 1.833 1.875 1.917
    x_dist   (x, y) float64 -4.659e+03 -4.659e+03 ... -3.76e+03 -3.76e+03
    y_dist   (x, y) float64 -523.5 -522.5 -521.5 -520.5 ... 373.5 374.5 375.5
    lat      (x, y) float64 46.95 46.95 46.95 46.96 ... 54.74 54.73 54.73 54.73
    lon      (x, y) float64 3.589 3.601 3.613 3.625 ... 15.66 15.67 15.69 15.7
Dimensions without coordinates: x, y
Data variables:
    radolan  (time, x, y) float64 -9.999e+03 -9.999e+03 ... -9.999e+03
Attributes:
    Conventions:  CF-1.10
    title:        DWD Climate Data Center (CDC): Recent hourly sliding RADOLA...
    institution:  Deutscher Wetterdienst (DWD)DWD Climate Data Center (CDC)
    source:       RADOLAN version: 2.29.1. The routine procedure RADOLAN (Rad...
    history:      2022-09-28 13:32:31.710520: data download and file creation...
    references:   Bartels, H. et al., 2004: Zusammenfassender Abschlussberich...
    comment:      **DATA ORIGIN:** Weather Radars can only measure the reflec...
    id:           urn:x-wmo:md:de.dwd.cdc::gridsgermany-daily-radolan-recent-bin